# Fetching Data from API

This file is contains all the necessary logic and functions to fetch data from OC, Wise, Stripe and PayPal.

In [2]:
import pandas as pd
import requests
import json
import time

pd.set_option('display.max_columns', None)

In [3]:
# host variables
host = 'opensource'
dateFrom = '2022-12-31T22:59:59.999Z'
dateTo = '2023-12-31T22:59:59.999Z'
year = '2023'

In [4]:
# load API tokens from .env file
import os
from dotenv import load_dotenv
load_dotenv()
octoken = os.getenv('OCTOKEN')
wisetoken = os.getenv('WISETOKEN')

# set the profile id of the target Wise profile
wiseprofile = os.getenv('WISEPROFILE')

# Get data

In [5]:
# load transactions from open collective or from local file

def fetch_transactions():
    url = f"https://opencollective.com/api/graphql/v2?personalToken={octoken}"

    # Your GraphQL query
    query = """
    query TransactionsTable(
        $hostAccount: AccountReferenceInput,
        $limit: Int!,
        $offset: Int!,
        $type: TransactionType,
        $paymentMethodType: [PaymentMethodType],
        $dateFrom: DateTime,
        $dateTo: DateTime,
        $kind: [TransactionKind],
        $includeIncognitoTransactions: Boolean,
        $includeGiftCardTransactions: Boolean,
        $includeChildrenTransactions: Boolean,
        $virtualCard: [VirtualCardReferenceInput],
        $orderBy: ChronologicalOrderInput,
        $group: String,
        $includeHost: Boolean,
        $expense: ExpenseReferenceInput,
        $order: OrderReferenceInput
      ) {
        transactions(
          host: $hostAccount
          limit: $limit
          offset: $offset
          type: $type
          paymentMethodType: $paymentMethodType
          dateFrom: $dateFrom
          dateTo: $dateTo
          kind: $kind
          includeIncognitoTransactions: $includeIncognitoTransactions
          includeGiftCardTransactions: $includeGiftCardTransactions
          includeChildrenTransactions: $includeChildrenTransactions
          includeDebts: true
          virtualCard: $virtualCard
          orderBy: $orderBy
          group: $group
          includeHost: $includeHost
          expense: $expense
          order: $order
        ) {
          ...TransactionsTableQueryCollectionFragment
          __typename
        }
      }
      
      fragment TransactionsTableQueryCollectionFragment on TransactionCollection {
        totalCount
        offset
        limit
        nodes {
          id
          uuid
          kind
          amount {
            currency
            valueInCents
            __typename
          }
          amountInHostCurrency {
            currency
            valueInCents
            __typename
          }
          netAmount {
            currency
            valueInCents
            __typename
          }
          netAmountInHostCurrency {
            currency
            valueInCents
            __typename
          }
          paymentProcessorFee {
            currency
            valueInCents
            __typename
          }
          paymentMethod {
            name
            service
            sourcePaymentMethod {
              id
              name
              service
              __typename
            }
            type
          }
          payoutMethod {
            type
            name
            type
          }
          giftCardEmitterAccount {  
            id
            name
            slug
            __typename
          }
          group
          type
          description
          createdAt
          merchantId
          isRefunded
          isRefund
          refundTransaction {
            id
          }
          isOrderRejected
          account {
            ... on AccountWithParent {
              parent {
                id
                slug
                name
              }
            }
            id
            legacyId
            name
            slug
            isIncognito
            type
            __typename
          }
          oppositeAccount {
            id
            legacyId
            name
            slug
            isIncognito
            type
            __typename
          }
          expense {
            id
            type
            description
            invoiceInfo 
            tags
            payee {
              id
              name
              slug
              type
              __typename
            }
            __typename
          }
          permissions {
            id
            canRefund
            canDownloadInvoice
            canReject
            __typename
          }
          __typename
        }
        __typename
      }
    """

    headers = {
        "Authorization": f"Bearer {octoken}",
        "Content-Type": "application/json"
    }

    # Initial variables setup
    variables = {
        "hostAccount": {"slug": host},
        "includeIncognitoTransactions": True,
        "includeChildrenTransactions": True,
        "limit": 10000,  # Adjust if necessary but keep a sensible number to avoid server strain
        "offset": 0,  # Will be adjusted for each subsequent request
        "dateFrom": dateFrom,
        "dateTo": dateTo,
        "orderBy": {"field": "CREATED_AT", "direction": "DESC"},
        "includeHost": True
    }

    all_transactions = []  # To hold all transactions
    while True:

        # Try catch around json_data = response.json() to avoid errors

        max_retries = 10  # Setting the maximum number of retries
        retries = 0  # Initial retry count
        response = ''

        while retries < max_retries:
            try:
                # Make the HTTP request
                response = requests.post(
                    url, json={'query': query, 'variables': variables}, headers=headers)
                json_data = response.json()
                break
            except Exception as e:
                print("Request failed. Attempt:", retries + 1)
                print(e)
                retries += 1
                time.sleep(10)

        if retries == max_retries:
            print("Maximum retry attempts reached. Exiting.")
        # Extract data
        transactions = json_data['data']['transactions']['nodes']
        all_transactions.extend(transactions)

        # Pagination: Update offset
        variables['offset'] += variables['limit']

        # Check if all transactions are fetched
        if len(all_transactions) >= json_data['data']['transactions']['totalCount']:
            break

        # print progress
        print(f'Fetched {len(all_transactions)} transactions')
        # sleep for 10 seconds to avoid server strain
        time.sleep(5)

    return all_transactions

# if account has property parent, replace account with parent
def replace_account_with_parent(transaction):
    if 'parent' in transaction['account']:
        transaction['account'] = transaction['account']['parent']
    return transaction

def post_process_transactions(all_transactions):
    return list(map(replace_account_with_parent, all_transactions))

all_transactions = fetch_transactions()
# dump all transactions to a json file
post_process_transactions(all_transactions)
with open(f'data/{host}/{year}/{host}_{year}_all_platform_transactions.json', 'w') as f:
  json.dump(all_transactions, f, indent=2)
# convert the json file to a dataframe
df_platform = pd.json_normalize(all_transactions)
# save dataframe as pickle
df_platform.to_pickle(
    f'data/{host}/{year}/df_{host}_{year}_all_platform_transactions.pkl')

Fetched 10000 transactions
Fetched 20000 transactions
Fetched 30000 transactions
Fetched 40000 transactions
Fetched 50000 transactions
Fetched 60000 transactions
Fetched 70000 transactions
Fetched 80000 transactions
Fetched 90000 transactions
Fetched 100000 transactions
Fetched 110000 transactions
Fetched 120000 transactions
Fetched 130000 transactions
Fetched 140000 transactions
Fetched 150000 transactions
Fetched 160000 transactions
Fetched 170000 transactions
Fetched 180000 transactions
Fetched 190000 transactions
Fetched 200000 transactions
Fetched 210000 transactions
Fetched 220000 transactions
Fetched 230000 transactions
Fetched 240000 transactions
Fetched 250000 transactions
Fetched 260000 transactions
Fetched 270000 transactions
Fetched 280000 transactions
Fetched 290000 transactions
Fetched 300000 transactions
Fetched 310000 transactions
Fetched 320000 transactions
Fetched 330000 transactions
Fetched 340000 transactions
Fetched 350000 transactions
Fetched 360000 transactions
F

In [6]:
# Define the base URL for the Wise API
BASE_URL = 'https://api.transferwise.com/v1/transfers'

def fetch_all_transfers(profile_id=None, status=None, source_currency=None, 
                        target_currency=None, created_date_start=None, 
                        created_date_end=None, limit=100):
    headers = {
        'Authorization': f'Bearer {wisetoken}'
    }
    offset = 0
    all_transfers = []

    while True:
        # Prepare query parameters
        query_params = {'limit': limit, 'offset': offset}
        if profile_id:
            query_params['profile'] = profile_id
        if status:
            query_params['status'] = status
        if source_currency:
            query_params['sourceCurrency'] = source_currency
        if target_currency:
            query_params['targetCurrency'] = target_currency
        if created_date_start:
            query_params['createdDateStart'] = created_date_start
        if created_date_end:
            query_params['createdDateEnd'] = created_date_end

        # Make the GET request to the Wise API
        response = requests.get(BASE_URL, headers=headers, params=query_params)
        data = response.json()

        print(data)

        # Break if there are no more results
        if not data:
            break

        for transfer in data:
            print(transfer)
            break; 
            all_transfers.append({
                'id': transfer['id'],
                'user': transfer['user'],
                'targetAccount': transfer['targetAccount'],
                'sourceAccount': transfer['sourceAccount'],
                'quote': transfer['quote'],
                'quoteUuid': transfer['quoteUuid'],
                'status': transfer['status'],
                'reference': transfer['reference'],
                'rate': transfer['rate'],
                'created': transfer['created'],
                'business': transfer['business'],
                'transferRequest': transfer['transferRequest'],
                'details.reference': transfer.get('details', {}).get('reference'),
                'hasActiveIssues': transfer['hasActiveIssues'],
                'sourceCurrency': transfer['sourceCurrency'],
                'sourceValue': transfer['sourceValue'],
                'targetCurrency': transfer['targetCurrency'],
                'targetValue': transfer['targetValue'],
                'customerTransactionId': transfer['customerTransactionId']
            })
        break;
        # Update the offset for the next iteration
        offset += limit

    return all_transfers

transfers = fetch_all_transfers(profile_id=wiseprofile, created_date_start=dateFrom, created_date_end=dateTo)
# create transfers dataframe
df_wise = pd.DataFrame()
for transfer in transfers:
    # add to dataframe with concat
    df_wise = pd.concat([df_wise, pd.DataFrame([transfer])])
# save dataframe as pickle
df_wise.to_pickle(f'data/{host}/{year}/df_{host}_{year}_wise_transactions.pkl')

[{'id': 575093811, 'user': 6421549, 'targetAccount': 298725929, 'sourceAccount': 59807023, 'quote': None, 'quoteUuid': 'be48f661-368c-430b-82f9-94a101390e59', 'status': 'outgoing_payment_sent', 'reference': '', 'rate': 1.0, 'created': '2023-01-02 00:07:08', 'business': 4556280, 'transferRequest': None, 'details': {'reference': ''}, 'hasActiveIssues': False, 'sourceCurrency': 'GBP', 'sourceValue': 46.86, 'targetCurrency': 'GBP', 'targetValue': 46.86, 'customerTransactionId': '72d112b5-775a-416e-a0d0-df6219c9d708'}, {'id': 575919595, 'user': 6421549, 'targetAccount': 219325936, 'sourceAccount': 59807023, 'quote': None, 'quoteUuid': '21416ea6-87fd-4bbd-8a64-2082c6894abc', 'status': 'outgoing_payment_sent', 'reference': '', 'rate': 1.0, 'created': '2023-01-02 23:47:59', 'business': 4556280, 'transferRequest': None, 'details': {'reference': ''}, 'hasActiveIssues': False, 'sourceCurrency': 'USD', 'sourceValue': 1550.0, 'targetCurrency': 'USD', 'targetValue': 1550.0, 'customerTransactionId': 